In [2]:
import os,sys
sys.path.insert(0,'../../libs')
import openai
from llm_utils import BSAgent
from data_utils import train_val_test_split,load_split_climate_data
from utils import download_hf_model
import pandas as pd
import re,json,copy
from tqdm import tqdm
from prompts import short_cot_pt,short_cot_pt_2label,long_cot_pt,long_cot_pt_2label,long_fewshotcot_pt_2label
import pprint

In [3]:
from pydantic import BaseModel
from typing import Literal

In [4]:
from dotenv import load_dotenv
env_path = '../../../.env'
load_dotenv(dotenv_path=env_path)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please check your .env file.")


In [5]:
data_folder = '/ephemeral/home/xiong/data/Fund/Climate'
# data_path = os.path.join(data_folder,'Climate training paragraphs.csv')
# ds = load_split_climate_data(data_path,merge_neutral=True,verbose=True)
# ds['test'].to_csv(data_folder+'/test.csv')
# ds['validation'].to_csv(data_folder+'/validation.csv')
# ds['train'].to_csv(data_folder+'/train.csv')

test_data = pd.read_csv(data_folder+'/test.csv')
val_data = pd.read_csv(data_folder+'/validation.csv')
train_data = pd.read_csv(data_folder+'/train.csv')

In [6]:
class ClimateClassification(BaseModel):
    justification: str
    classification: Literal["favorable", "unfavorable", "neutral"]

class ClimateClassification_2label(BaseModel):
    justification: str
    classification: Literal["favorable", "unfavorable"]


In [8]:
## download models
model_name_list = ['Qwen/Qwen2.5-7B-Instruct','Qwen/Qwen2.5-32B-Instruct',
                   'meta-llama/Llama-3.1-8B-Instruct','meta-llama/Llama-3.1-70B-Instruct',
                   'deepseek-ai/DeepSeek-R1-Distill-Qwen-32B','deepseek-ai/DeepSeek-R1-Distill-Llama-70B',
                   'Qwen/QwQ-32B','deepseek-ai/DeepSeek-R1-Distill-Llama-8B']
for model_name in model_name_list:
    # Create the target directory path
    target_dir = '/ephemeral/home/xiong/data/hf_cache/' + model_name
    # Check if model already exists before downloading
    if os.path.exists(target_dir) and os.listdir(target_dir):
        print(f"Model {model_name} already exists at {target_dir}, skipping download")
    else:
        print(f"Downloading model {model_name}...")
        download_hf_model(model_name, target_dir, hf_token=os.getenv('huggingface_token'))

Model Qwen/Qwen2.5-7B-Instruct already exists at /ephemeral/home/xiong/data/hf_cache/Qwen/Qwen2.5-7B-Instruct, skipping download
Model Qwen/Qwen2.5-32B-Instruct already exists at /ephemeral/home/xiong/data/hf_cache/Qwen/Qwen2.5-32B-Instruct, skipping download
Model meta-llama/Llama-3.1-8B-Instruct already exists at /ephemeral/home/xiong/data/hf_cache/meta-llama/Llama-3.1-8B-Instruct, skipping download
Model meta-llama/Llama-3.1-70B-Instruct already exists at /ephemeral/home/xiong/data/hf_cache/meta-llama/Llama-3.1-70B-Instruct, skipping download
Model deepseek-ai/DeepSeek-R1-Distill-Qwen-32B already exists at /ephemeral/home/xiong/data/hf_cache/deepseek-ai/DeepSeek-R1-Distill-Qwen-32B, skipping download


Fetching 26 files: 100%|██████████| 26/26 [02:49<00:00,  6.51s/it]


Model Qwen/QwQ-32B already exists at /ephemeral/home/xiong/data/hf_cache/Qwen/QwQ-32B, skipping download


Fetching 11 files: 100%|██████████| 11/11 [00:45<00:00,  4.15s/it]


In [7]:
# use openap modesl 
# agent = BSAgent(model='gpt-4o-mini')
# Try other opens rousce modesl 
# python -m vllm.entrypoints.openai.api_server --model /home/xiong/data/hf_cache/llama-3.1-8B-Instruct --dtype auto --servered_model_name llama-3.1-8b-Instruct
agent = BSAgent(base_url='http://localhost:8100/v1',api_key='abc')
# agent.model = agent.client.models.list().data[0].id
# print(agent.model) 
agent.connection_test()

The capital of France is Paris.


'The capital of France is Paris.'

In [8]:
def get_climate_classifications(agent, dataset, prompt_template):
    results = []
    for i in tqdm(range(len(dataset))):
        structured_prompt = copy.deepcopy(prompt_template)
        structured_prompt['user'] = structured_prompt['user'].format(PARAGRAPH=dataset.iloc[i].paragraph)
        try:
            response = agent.get_response_content(prompt_template=structured_prompt, response_format=ClimateClassification)
            results.append({
                'paragraph': dataset.iloc[i].paragraph,
                'true_label': dataset.iloc[i].label,
                'predicted_label': response.classification,
                'justification': response.justification
            })
        except Exception as e:
            print(f"Error processing row {i}: {str(e)}")
            results.append({
                'paragraph': dataset.iloc[i].paragraph,
                'true_label': dataset.iloc[i].label,
                'predicted_label': None,
                'justification': f"Error: {str(e)}"
            })
    return pd.DataFrame(results)

In [17]:
long_fewshotcot_pt_2label = {
    'system':
        """You are an economist analyzing newspaper paragraphs about climate issues. For each paragraph, classify it as one of the following:
            1. **favorable**: supports or promotes actions, policies, or economic measures that mitigate climate change or transition to sustainable practices.
            2. **unfavorable**: undermines or criticizes climate-friendly policies, denies climate change, or argues against sustainability measures.  
            
        For each paragraph, provide a brief justification for your classification.
        
        **Here are few examples:**
        ----
        **Statement:**
        But it will be scrutinised in minute detail by envoys from poorer countries who say they cannot sign up to a deal in Paris if it lacks the funding they need to shift to greener energy systems and deal with the floods and heatwaves that scientists say are likely to increase as the climate changes.
        **Return:**
        ```json
        {
        "justification": "While the statement acknowledges the importance of climate action, it focuses on potential obstacles, by highlighting that poorer countries “cannot sign up” without this financial support.",
        "classification": "unfavorable",
        }```
        
        **Statement:**
        Matthew Gray at Carbon Tracker, a think-tank, said the price of carbon credits was being supported by the gradual reopening of economies and expectations that industrial activity, and emissions, will rebound in the coming months. '[Carbon] has been the number one performer in the European energy complex for some time now and is being bolstered by hopes of trade relief and an easing of lockdown restrictions,' Mr Gray said.
        **Return:**
        ```json
        {
        "justification": "It highlights sustained market support for carbon pricing, which can incentivize lower emissions.",
        "classification": "favorable",
        }```
        ----
        
        Please respond in clean json format as follow and your output should include only this dictionary, with no additional commentary.
        ```json
        {
        "justification": "<brief reason>"
        "classification": "<favorable | unfavorable>"
        }```
        """,
    'user':
        """ Please Read carefully andclassify the following paragraph.
        Here is the paragraph: 
        ----
        ----
        {PARAGRAPH} 
        ----
        ----
        
        **Additional Tips:**
        1. Look for keywords: Words such as “subsidies,” “carbon taxes,” “renewable energy,” “sustainability,” etc., can indicate climate-favorable sentiments if they are portrayed in a positive light. 
        Conversely, mentions of “regulatory burdens,” “economic drawbacks,”, "economic cost" or “inefficiency” tied to climate policies may indicate a climate-unfavorable stance.
        2. Any statement complains about current situation, delay of climate policy action, or stating the current situation is bad. We put them as negative. 
        3. Focus on identifying the underlying viewpoint regarding climate-related actions or policies, especially from an economic standpoint.
        4. If you think the paragraph is neutral, please put it as favorable.
        
        Please respond in clean json format, it should include only this dictionary, with no additional commentary.
        """
    }

In [19]:
long_fewshotcot_pt_2label = {
    'system':
        """You are an economist analyzing newspaper paragraphs about climate issues. For each paragraph, classify it as one of the following:
            1. **favorable**: supports or promotes actions, policies, or economic measures that mitigate climate change or transition to sustainable practices.
            2. **unfavorable**: undermines or criticizes climate-friendly policies, denies climate change, or argues against sustainability measures.  
            
        For each paragraph, provide a brief justification for your classification.
        
        **Here are few examples:**
        ----
        **Statement:**
        But it will be scrutinised in minute detail by envoys from poorer countries who say they cannot sign up to a deal in Paris if it lacks the funding they need to shift to greener energy systems and deal with the floods and heatwaves that scientists say are likely to increase as the climate changes.
        **Return:**
        ```json
        {
        "justification": "While the statement acknowledges the importance of climate action, it focuses on potential obstacles, by highlighting that poorer countries “cannot sign up” without this financial support.",
        "classification": "unfavorable"
        }```
        
        **Statement:**
        Matthew Gray at Carbon Tracker, a think-tank, said the price of carbon credits was being supported by the gradual reopening of economies and expectations that industrial activity, and emissions, will rebound in the coming months. '[Carbon] has been the number one performer in the European energy complex for some time now and is being bolstered by hopes of trade relief and an easing of lockdown restrictions,' Mr Gray said.
        **Return:**
        ```json
        {
        "justification": "It highlights sustained market support for carbon pricing, which can incentivize lower emissions.",
        "classification": "favorable"
        }```
        ----
        
        **Return the response in valid JSON** with the following structure , with no additional commentary:**
        ```json
        {
        "justification": "<brief reason>"
        "classification": "<favorable | unfavorable>"
        }```
        """,
    'user':
        """ Please Read carefully andclassify the following paragraph.
        Here is the paragraph: 
        ----
        ----
        {PARAGRAPH} 
        ----
        ----
        
        **Additional Tips:**
        1. Look for keywords: Words such as “subsidies,” “carbon taxes,” “renewable energy,” “sustainability,” etc., can indicate climate-favorable sentiments if they are portrayed in a positive light. 
        Conversely, mentions of “regulatory burdens,” “economic drawbacks,”, "economic cost" or “inefficiency” tied to climate policies may indicate a climate-unfavorable stance.
        2. Any statement complains about current situation, delay of climate policy action, or stating the current situation is bad. We put them as negative. 
        3. Focus on identifying the underlying viewpoint regarding climate-related actions or policies, especially from an economic standpoint.
        4. If you think the paragraph is neutral, please put it as favorable.
        """
    }

In [21]:
# Get predictions for validation and test sets
val_results = get_climate_classifications(agent, val_data, long_fewshotcot_pt_2label)
val_results.to_csv(data_folder+'/val_results_v2.csv')

print("\nValidation Results:")
print(f"Total samples: {len(val_results)}")
print(f"Successfully processed: {len(val_results[val_results.predicted_label.notna()])}")
val_accuracy = (val_results['true_label'] == val_results['predicted_label']).mean()
print(f"Validation Accuracy: {val_accuracy:.2%}")


100%|██████████| 108/108 [01:03<00:00,  1.70it/s]


Validation Results:
Total samples: 108
Successfully processed: 108
Validation Accuracy: 80.56%


#### Try run with asyc clent

In [20]:
# Run the test
import nest_asyncio
import asyncio
nest_asyncio.apply()
from llm_utils_async import AsyncBSAgent


In [ ]:
agent = AsyncBSAgent(model='llama-3.1-8b-Instruct',base_url='http://localhost:8800/v1',api_key='abc')
print(agent.model) 

In [25]:
async def get_climate_classifications(agent, dataset, prompt_template):
    async def process_row(i):
        structured_prompt = copy.deepcopy(prompt_template)
        structured_prompt['user'] = structured_prompt['user'].format(PARAGRAPH=dataset.iloc[i].paragraph)
        try:
            response = await agent.get_response_content(prompt_template=structured_prompt, response_format=ClimateClassification)
            return {
                'paragraph': dataset.iloc[i].paragraph,
                'true_label': dataset.iloc[i].label,
                'predicted_label': response.classification,
                'justification': response.justification
            }
        except Exception as e:
            print(f"Error processing row {i}: {str(e)}")
            return {
                'paragraph': dataset.iloc[i].paragraph,
                'true_label': dataset.iloc[i].label,
                'predicted_label': None,
                'justification': f"Error: {str(e)}"
            }

    tasks = [process_row(i) for i in range(len(dataset))]
    results = await asyncio.gather(*tasks)
    return pd.DataFrame(results)

In [ ]:
val_results = asyncio.run(get_climate_classifications(agent, train_data, long_fewshotcot_pt_2label))
print("\nValidation Results:")
print(f"Total samples: {len(val_results)}")
print(f"Successfully processed: {len(val_results[val_results.predicted_label.notna()])}")
val_accuracy = (val_results['true_label'] == val_results['predicted_label']).mean()
print(f"Validation Accuracy: {val_accuracy:.2%}")